## Conference

In [33]:
# codes are from https://github.com/tamlhp/template/blob/master/dblp/download_paperinfo_from_dblp.ipynb
# with lil bit of revision
import pandas as pd
import urllib.request
import shutil
import re
import os
import pathlib
import argparse
import pdb
import threading
from bs4 import BeautifulSoup

def site_open(site):
    '''Makes connection and opens up target website. Returns a website object.'''
    try:
        #sets up request object
        req = urllib.request.Request(site)

        #adds User-Agent info to request object
        req.add_header("User-Agent","Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) \
         AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.46 Safari/536.5")

        #opens up site
        website = urllib.request.urlopen(req)

        return website
    except urllib.request.URLError:
        print('Could not connect to '+ site + '!')
        pass
    
def soup_site(site):
    '''opens site and turns it into a format to easily parse the DOM. Returns a Soup Object'''
    try:
        website = site_open(site)
#         return BeautifulSoup(website, "html5lib")
        return BeautifulSoup(website, "lxml")
    except Exception as e:
        print(e)
        return -1

print("Initialization Done")

Initialization Done


In [12]:
import re

# conf_name = 'cscw'
conf_name = 'wsdm'

year = 2022
#year = 2021

#soup = soup_site("https://dblp.org/db/conf/cscw/cscw{}c.html".format(year))#CSCW
soup = soup_site("https://dblp.org/db/conf/wsdm/wsdm{}.html".format(year))#WSDM
    
if soup != -1:
    title = soup.find("li", {"class" : "entry inproceedings"}).find("cite", {"class" : "data tts-content"}).find("span", {"class" : "title"}).text
    authors = [t.text for t in soup.find("li", {"class" : "entry inproceedings"}).find("cite", {"class" : "data tts-content"}).findAll("span", {"itemprop" : "author"})]
    year = soup.find("header", {"id": "headline"}).find("h1").text
    doi = soup.find("li", {"class" : "entry inproceedings"}).find("div", {"class" : "head"}).find("a")['href']
    print(authors, title, year, doi)

['Ricardo Baeza-Yates'] Ethical Challenges in AI. 14th WSDM 2022:
Virtual Event / Tempe, AZ, USA https://doi.org/10.1145/3488560.3498370


In [51]:
import csv

#conf_name = 'cscw'
#conf_name = 'wsdm'
#conf_name = 'chi'
#conf_name = 'iswc'
conf_name = 'ubicomp'

#year = 2022
#year = 2021
year = 2020

#volume = "https://dblp.org/db/conf/wsdm/wsdm{}.html".format(year)
#volume = "https://dblp.org/db/conf/cscw/cscw{}c.html".format(year)#CSCW
#volume = "https://dblp.org/db/conf/chi/chi{}.html".format(year)#CHI
#volume = "https://dblp.org/db/conf/iswc/iswc{}.html".format(year)#ISWC
#volume = "https://dblp.org/db/conf/huc/ubicomp{}ap.html".format(year)#UbiComp2021
#volume = "https://dblp.org/db/conf/iswc/iswc{}.html".format(year)#ISWC2020
volume = "https://dblp.org/db/conf/huc/ubicomp{}ap.html".format(year)#UbiComp2020

f = open('crawled_csv_files/{}{}.csv'.format(conf_name,year), 'w', newline='')

writer = csv.writer(f, delimiter="\t")

soup = soup_site(volume)

if soup != -1:
    year = re.findall(r'\d+', soup.find("header", {"id": "headline"}).find("h1").text)[0]

    for paper in soup.findAll("li", {"class" : "entry inproceedings"}):
        try:
            doi = paper.find("div", {"class" : "head"}).find("a")['href']
        except:
            doi = "N/A"

        data = paper.find("cite", {"class" : "data tts-content"})
        if data is None:
            data = paper.find("article", {"class" : "data"})
        if data is None:
            data = paper.find("cite", {"class" : "data"})
        authors = [t.text for t in data.findAll("span", {"itemprop" : "author"})]
        #authors = ','.join(authors)
        title = data.find("span", {"class" : "title"}).text
        writer.writerow([", ".join(authors), title, year, doi])

    f.close()
print("Done")

Done


## Journals

In [44]:
import csv

#journal_name = 'snam'
#journal_name = 'tmm'
#journal_name = 'pami'

#year = 2023
#year = 2022
year = 2021

# volume = "https://dblp.org/db/journals/snam/snam12.html"#SNAM2022
# volume = "https://dblp.org/db/journals/snam/snam11.html"#SNAM2021
# volume = "https://dblp.org/db/journals/tmm/tmm24.html"#TMM2022
# volume = "https://dblp.org/db/journals/tmm/tmm23.html"#TMM2021
# volume = "https://dblp.org/db/journals/pami/pami45.html"#PAMI2023
# volume = "https://dblp.org/db/journals/pami/pami44.html"#PAMI2022


f = open('crawled_csv_files/{}{}.csv'.format(journal_name,year), 'w', newline='')

writer = csv.writer(f, delimiter="\t")
    
soup = soup_site(volume)

if soup != -1:
    h2sections = soup.findAll("h2")
    year = 0
    for h2section in h2sections:
        match = re.match(r'.*([1-3][0-9]{3})', h2section.text)
        if match is not None:
            year = match.group(1)
            break  
    print(year)

    for j, section in enumerate(soup.findAll("ul", {"class" : "publ-list"})):
        for paper in section.findAll("li", {"class" : "entry article"}):
            try:
                doi = paper.find("nav", {"class" : "publ"}).find("div", {"class" : "head"}).find("a")['href']
            except:
                doi = "error"
            data = paper.find("div", {"class" : "data"})
            if data is None:
                data = paper.find("article", {"class" : "data"})
            if data is None:
                data = paper.find("cite", {"class" : "data"})
            authors = [t.text for t in data.findAll("span", {"itemprop" : "author"})]
            title = data.find("span", {"class" : "title"}).text.encode("utf-8")
            writer.writerow([", ".join(authors).encode("utf-8"), title, year, doi])

    f.close()
print("Done")

0
Done
